![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/Spark%20v2.7.6%20Notebooks/8.Generic_Classifier.ipynb)

# 8. Generic Classifier

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

In [ ]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh -p 2.4.4

In [ ]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print (sparknlp.version())
print (sparknlp_jsl.version())

2.7.4
2.7.6


In [ ]:
 # if you want to start the session with custom params as in start function above

def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()


#spark = start(secret)

In [ ]:
spark

## load dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv

In [ ]:
import pandas as pd
dataframe = pd.read_csv('petfinder-mini.csv')

In [ ]:
# In the original dataset "4" indicates the pet was not adopted.
import numpy as np
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [ ]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [ ]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,1


In [ ]:
dataframe.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'target'],
      dtype='object')

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11537 non-null  object
 1   Age           11537 non-null  int64 
 2   Breed1        11537 non-null  object
 3   Gender        11537 non-null  object
 4   Color1        11537 non-null  object
 5   Color2        11537 non-null  object
 6   MaturitySize  11537 non-null  object
 7   FurLength     11537 non-null  object
 8   Vaccinated    11537 non-null  object
 9   Sterilized    11537 non-null  object
 10  Health        11537 non-null  object
 11  Fee           11537 non-null  int64 
 12  Description   11537 non-null  object
 13  PhotoAmt      11537 non-null  int64 
 14  target        11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [ ]:
dataframe.target.value_counts()

1    8457
0    3080
Name: target, dtype: int64

In [ ]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [ ]:
y.nunique()

2

In [ ]:
X.shape

(11537, 302)

In [ ]:
input_dim = X.shape[1]

In [ ]:
input_dim

302

In [ ]:
df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df['target']= y

df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.220992,0.151594,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.144907,0.000000,0.0,0.158869,0.0,0.0,0.000000,0.170202,0.000000,0.192465,0.190783,0.000000,0.132427,0.176639,0.151246,0.000000,0.0,0.000000,0.000000,0.257843,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.256152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250843,0.000000,0.0,0.000000,0.000000,0.213817,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.307225,0.0,0.178193,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.210999,0.209155,0.209619,0.000000,0.000000,0.414527,0.179294,0.0,0.000000,0.000000,0.141336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.257886,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.198538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.081638,0.000000,0.105362,0.128186,0.0,0.000000,0.0,0.000000,0.107062,0.086649,0.0,0.000000,0.0,0.0,0.000000,0.125751,0.218471,0.142200,0.000000,0.000000,0.195683,0.000000,0.111746,0.120832,0.0,0.120993,0.000000,0.000000,0.223069,0.117846,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [ ]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|col_293|col_294|col_295|col_296|col_297|col_298|            col_299|             col_300|            col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|-0.4524794726808656|  0.9502875792756131|-0.8297616989552165|     1|
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844|     1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows



In [ ]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9230
Test Dataset Count: 2307


## Create a custom DL architecture with TF

In [ ]:
from sparknlp_jsl.training import tf_graph
%tensorflow_version 1.x
!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")


TensorFlow 1.x selected.
generic_classifier parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [ ]:
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",build_params=DL_params, model_location="/content/gc_graph", model_filename="auto")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
generic_classifier graph exported to /content/gc_graph/gcl.302.2.pb


In [ ]:
#or just use the one we already have in the rpo

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /content/gc_graph

In [ ]:
from sparknlp_jsl.base import *

!mkdir logs

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('/content/gc_graph/gcl.302.2.pb')\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])



In [ ]:
%%time

# train 50 epochs (takes around 45 seconds)

clf_model = clf_Pipeline.fit(training_data)


CPU times: user 314 ms, sys: 33.5 ms, total: 348 ms
Wall time: 48.2 s


In [ ]:
! cd logs && ls

GenericClassifierApproach_f9a33df28972.log


In [ ]:
!cat logs/GenericClassifierApproach_f9a33df28972.log

Training 50 epochs
Epoch 1/50	0.87s	Loss: 18.95888	ACC: 0.6292986	Validation ACC: 0.6928494
Epoch 2/50	0.63s	Loss: 16.505543	ACC: 0.700251	Validation ACC: 0.69176596
Epoch 3/50	0.63s	Loss: 14.912577	ACC: 0.74027646	Validation ACC: 0.6836403
Epoch 4/50	0.63s	Loss: 13.553513	ACC: 0.7750257	Validation ACC: 0.6966414
Epoch 5/50	0.64s	Loss: 11.651418	ACC: 0.81261265	Validation ACC: 0.7307692
Epoch 6/50	0.64s	Loss: 10.084301	ACC: 0.8511584	Validation ACC: 0.7269772
Epoch 7/50	0.63s	Loss: 8.374009	ACC: 0.8766858	Validation ACC: 0.7475623
Epoch 8/50	0.61s	Loss: 6.7876487	ACC: 0.9121748	Validation ACC: 0.73510295
Epoch 9/50	0.61s	Loss: 5.4898	ACC: 0.93192405	Validation ACC: 0.7378115
Epoch 10/50	0.60s	Loss: 4.117794	ACC: 0.953166	Validation ACC: 0.7345612
Epoch 11/50	0.62s	Loss: 3.0352137	ACC: 0.9671943	Validation ACC: 0.75352114
Epoch 12/50	0.61s	Loss: 2.2392647	ACC: 0.9789768	Validation ACC: 0.7513543
Epoch 13/50	0.64s	Loss: 1.7917055	ACC: 0.9844342	Validation ACC: 0.75189596
Epoch 14/50	0.62

In [ ]:
pred_df = clf_model.transform(test_data)

In [ ]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [0]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     0|   [0]|
|     0|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [0]|
|     1|   [0]|
|     1|   [1]|
|     1|   [0]|
|     1|   [0]|
|     0|   [0]|
+------+------+
only showing top 20 rows



In [ ]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [ ]:
preds_df

,target,result
0,1,1
1,1,0
2,1,1
3,1,0
4,1,0
...,...,...
2302,1,1
2303,1,0
2304,0,1
2305,0,1


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))


              precision    recall  f1-score   support

           0     0.4936    0.4483    0.4699       687
           1     0.7748    0.8049    0.7896      1620

    accuracy                         0.6987      2307
   macro avg     0.6342    0.6266    0.6297      2307
weighted avg     0.6911    0.6987    0.6944      2307

0.6987429562201994


## get prediction for random input

In [ ]:
pd.DataFrame([dataframe.loc[5191].to_dict()])

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Dog,2,Mixed Breed,Female,Black,Brown,Medium,Medium,Yes,Yes,Healthy,10,Friendly and pampered female puppy looking for...,2,1


In [ ]:
input_X = column_trans.transform(pd.DataFrame([dataframe.loc[0].to_dict()]).drop(['target'], axis=1))

input_y = dataframe.target[0]


In [ ]:
input_df = pd.DataFrame.sparse.from_spmatrix(input_X)

input_df.columns = ['col_{}'.format(i) for i in range(input_X.shape[1])]

input_df['target']= input_y

input_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.220992,0.151594,0.0,0.0,0.0,0.0,0.0,0.0,0.144907,0.0,0.0,0.158869,0.0,0.0,0.0,0.170202,0.0,0.192465,0.190783,0.0,0.132427,0.176639,0.151246,0.0,0.0,0.0,0.0,0.257843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.452479,0.950288,-0.829762,1


In [ ]:
input_spark_df = spark.createDataFrame(input_df)
input_spark_df.show(2)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [ ]:
clf_model.transform(input_spark_df).select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
+------+------+



# Case Study: Alexa Review Classification

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv

In [ ]:
import pandas as pd
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [ ]:
df.verified_reviews = df.verified_reviews.str.lower()

In [ ]:
df.feedback.value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


column_trans = make_column_transformer((OneHotEncoder(), ['rating','variation']),
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [ ]:
sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf['feedback']= y

sdf.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_982,col_983,col_984,col_985,col_986,col_987,col_988,col_989,col_990,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,col_1000,col_1001,col_1002,col_1003,col_1004,col_1005,col_1006,col_1007,col_1008,col_1009,col_1010,col_1011,col_1012,col_1013,col_1014,col_1015,col_1016,col_1017,col_1018,col_1019,col_1020,feedback
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145963,0.146283,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.16371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.104466,0.0,0.0,0.0,0.0,0.0,0.210616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
input_spark_df = spark.createDataFrame(sdf)

In [ ]:
input_spark_df.show(5)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+-------------------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------------------+------------------+------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------

In [ ]:

(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2506
Test Dataset Count: 644


In [ ]:
from sparknlp_jsl.base import *

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('/content/gc_graph/pet.in1202D.out2.pb')\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [ ]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['feedback']))


              precision    recall  f1-score   support

           0     0.8269    0.8958    0.8600        48
           1     0.9916    0.9849    0.9882       596

    accuracy                         0.9783       644
   macro avg     0.9092    0.9404    0.9241       644
weighted avg     0.9793    0.9783    0.9787       644

0.9782608695652174
